In [21]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [22]:
! dvc pull

/prepared/test.tsv    1.47M/4.54M [00:01<00:09,     358kB/s]





  7%|▋         |data/data.xml             2.44M/36.1M [00:01<01:06,     528kB/s]


 49%|████▊     |model.pkl                 1.23M/2.54M [00:01<00:03,     358kB/s]
  9%|▊         |data/prepared/train.tsv   1.58M/18.2M [00:01<00:48,     358kB/s]

 36%|███▌      |data/prepared/test.tsv    1.62M/4.54M [00:01<00:06,     461kB/s]




 57%|█████▋    |data/features/test.pkl     792k/1.35M [00:01<00:02,     209kB/s]



 11%|█▏        |data/features/train.pkl    624k/5.41M [00:01<00:26,     187kB/s]





  8%|▊         |data/data.xml             2.71M/36.1M [00:01<00:51,     681kB/s]
 10%|▉         |data/prepared/train.tsv   1.78M/18.2M [00:01<00:36,     476kB/s]


 55%|█████▌    |model.pkl                 1.41M/2.54M [00:01<00:02,     462kB/s]




 63%|██████▎   |data/features/test.pkl     872k/1.35M [00:01<00:01,     268kB/s]

 39%|███▉      |data/prepared/test.tsv    1.77M/4.54M [00:01<00:05,     569kB/s]



 13%|█▎        |da

## Easily reproduce somewhere else

### from cache

In [23]:
! pipenv run dvc repro

Loading .env environment variables...
If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>Stage 'data/data.xml.dvc' didn't change, skipping
Stage 'prepare' didn't change, skipping
Stage 'featurize' didn't change, skipping
Stage 'train' didn't change, skipping
Stage 'evaluate' didn't change, skipping
Data and pipelines are up to date.


### or force actual pipeline run

In [24]:
! dvc repro -f

If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>Verifying data sources in stage: 'data/data.xml.dvc'

Running stage 'prepare' with command:
	python src/prepare.py data/data.xml

Running stage 'featurize' with command:
	python src/featurization.py data/prepared data/features
The input data frame data/prepared/train.tsv size is (20017, 3)
The output matrix data/features/train.pkl size is (20017, 502) and data type is float64
The input data frame data/prepared/test.tsv size is (4983, 3)
The output matrix data/features/test.pkl size is (4983, 502) and data type is float64
  0% Saving features|                          |0.00/2.00 [00:00<?,     ?file/s]
Running stage 'train' with command:
	python src/train.py data/features model.pkl
Input matrix size (20017, 502)
X matrix size (20017, 500)
Y matrix size (20017,)
If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>
Running stage 'evaluate' with command:
	python src/evaluate.py model.pkl data/features sc

In [25]:
! dvc diff

If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>

### Assert equivalent result

In [26]:
! cat scores.json

{"auc": 0.5417487597055675}

### Now I can easily
* Making updates to the models
* see differences in parameters or results
* collaborate with others
* reproduce results

### Go and modify some parameters

In [27]:
! dvc status

Data and pipelines are up to date.


In [28]:
! dvc params diff

If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>

In [29]:
! dvc repro

If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>Stage 'data/data.xml.dvc' didn't change, skipping
Stage 'prepare' didn't change, skipping
Stage 'featurize' didn't change, skipping
Stage 'train' didn't change, skipping
Stage 'evaluate' didn't change, skipping
Data and pipelines are up to date.


In [30]:
! dvc metrics diff

If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>

In [31]:
! cat scores.json

{"auc": 0.5417487597055675}

In [32]:
! dvc plots diff -x recall -y precision

file:///home/nichita/Projects/dvc_intro/plots.html


## For reproducibility of this notebook:
### Remove tracked data from dvc (local and remote) and
### Remove unnecessary files 

In [33]:
! dvc remove data/data.xml.dvc
! rm data/data.xml

If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>

In [34]:
! dvc diff

If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>Deleted:
    data/data.xml

files summary: 0 added, 1 deleted, 0 modified


In [35]:
! git add data/data.xml.dvc
! rm -rf data/
! rm plots.html
! rm prc.json
! rm model.pkl

! git commit -m 'removes data'
! git push origin tutorial
! dvc push

[data_pipelines fcdbf05] removes data
 1 file changed, 3 deletions(-)
 delete mode 100644 data/data.xml.dvc
Everything up-to-date
Everything is up to date.


In [36]:
! echo "y" | dvc gc -cw

  0% Querying cache in 'gs://dvc_intro'|  |1.00/4.10k [00:00<10:52,  6.28file/s]

In [37]:
! echo 'y' | dvc destroy

This will destroy all information about your pipelines, all data files, as well as cache in .dvc/cache.
If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>

In [39]:
! git add .
! git commit -m "clean"
! git push origin tutorial